In [4]:
import pandas as pd
import numpy as np 
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler

# 키, 몸무게, 레이블이 적힌 CSV 파일 읽어 들이기
csv = pd.read_csv("bmi.csv")


scaler = MinMaxScaler()
wh =scaler.fit_transform(csv.loc[:,["weight","height"]])


# # 데이터 정규화
# csv["height"] = csv["height"] / 200
# csv["weight"] = csv["weight"] / 100
csv["height"]= wh[:,1]
csv["weight"]= wh[:,0] 


# 레이블을 배열로 변환하기
# - thin=(1,0,0) / normal=(0,1,0) / fat=(0,0,1)
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

# 테스트를 위한 데이터 분류
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

# 데이터 플로우 그래프 구축하기
# 플레이스홀더 선언하기
x  = tf.placeholder(tf.float32, [None, 2]) # 키와 몸무게 데이터 넣기
y_ = tf.placeholder(tf.float32, [None, 3]) # 정답 레이블 넣기

# 변수 선언하기
W1 = tf.Variable(tf.zeros([2, 3])); # 가중치
b1 = tf.Variable(tf.zeros([3])); # 바이어스
model = tf.matmul(x, W1) + b1

# 소프트맥스 회귀 정의하기
h = tf.nn.softmax(model)

# 모델 훈련하기
cross_entropy = -tf.reduce_sum(y_ * tf.log(h))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 정답률 구하기
predict = tf.equal(tf.argmax(h, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

# 세션 시작하기
sess = tf.Session() 
sess.run(tf.global_variables_initializer()) # 변수 초기화하기
 
# 학습시키기
for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i + 100]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "cre=", cre, "acc=", acc)
        
# 최종적인 정답률 구하기
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("정답률 =", acc)

Instructions for updating:
Colocations handled automatically by placer.


A:\Python\Anaconda3.5\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


step= 0 cre= 106.8153 acc= 0.6836
step= 500 cre= 33.44731 acc= 0.9758
step= 1000 cre= 24.252144 acc= 0.9796
step= 1500 cre= 23.432322 acc= 0.9798
step= 2000 cre= 18.470055 acc= 0.9818
step= 2500 cre= 18.270721 acc= 0.9812
step= 3000 cre= 13.019936 acc= 0.9798
정답률 = 0.9822
